In [6]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    # reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)

response = llm.invoke("The first person to land moon was..")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [ ]:
from langchain_community.document_loaders import WebBaseLoader  # Web scraping function

loader = WebBaseLoader("https://www.talentmate.com/jobs/uae/abu-dhabi/consultant-ai-software-engineer-digital-faas-abu-dhabi/2508-1998-1591")

In [9]:
page_data = loader.load().pop().page_content

In [11]:
print(page_data)




Consultant - AI Software Engineer - Digital FAAS - Abu Dhabi Jobs in Abu Dhabi, United Arab Emirates


































































































Home




											Jobs
										


Search Jobs



										Popular Searches
									

IT Jobs in Dubai

Engineering Jobs in Dubai

Accountant Jobs in Dubai

Sales Jobs






										Jobs by Categories
									

HR Jobs 

Marketing Jobs 

Customer Service Jobs 

Banking Jobs  


View All





										Jobs by Location
									

Jobs in
															UAE
														

Jobs in Dubai

Jobs in Abu Dhabi 

Jobs in Sharjah  


View All





										Jobs in Demand
									

Full-Time Jobs
											

Contract Jobs

Part-Time jobs
											

Freelance Jobs 

Temporary Jobs 







Premium Jobs


Articles



							Register CV
						




									CANDIDATE
								





                                        EMPLOYER
								











Back
                        to listing



In [13]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

[
  {
    "role": "Consultant - AI Software Engineer - Digital FAAS - Abu Dhabi",
    "experience": "3 to 4 years of related work experience",
    "skills": [
      "Practical experience in AI and ML development with Python, TensorFlow, PyTorch, and cloud platforms (Azure, AWS, GCP)",
      "Hands-on experience building and fine-tuning LLMs, working with frameworks like LangChain, HuggingFace, or OpenAI APIs",
      "Experience designing MLOps pipelines, AI solution architecture, and deploying AI models in production",
      "Exposure to GenAI use cases such as document summarization, chatbots, intelligent assistants, and code generation",
      "Domain knowledge in one or more of the following: Government, Healthcare, Financial Services, Retail, Telecom",
      "Strong ability to translate complex business problems into AI solutions with measurable impact",
      "Knowledge of data privacy, ethical AI, and responsible AI principles"
    ],
    "description": "As an AI Engineering prof

In [14]:
type(res.content)  # should convert to json

str

In [15]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Consultant - AI Software Engineer - Digital FAAS - Abu Dhabi',
  'experience': '3 to 4 years of related work experience',
  'skills': ['Practical experience in AI and ML development with Python, TensorFlow, PyTorch, and cloud platforms (Azure, AWS, GCP)',
   'Hands-on experience building and fine-tuning LLMs, working with frameworks like LangChain, HuggingFace, or OpenAI APIs',
   'Experience designing MLOps pipelines, AI solution architecture, and deploying AI models in production',
   'Exposure to GenAI use cases such as document summarization, chatbots, intelligent assistants, and code generation',
   'Domain knowledge in one or more of the following: Government, Healthcare, Financial Services, Retail, Telecom',
   'Strong ability to translate complex business problems into AI solutions with measurable impact',
   'Knowledge of data privacy, ethical AI, and responsible AI principles'],
  'description': 'As an AI Engineering professional, you will help businesses unlock va

In [16]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [ ]:
import chromadb 
import uuid

client = chromadb.PersistentClient('vectorstore') # to save the vector store db in the folder in local disk
collection = client.get_or_create_collection(name="portfolio")

# When the db is empty, we add the each row in csv file to vector db as a document and its metadata
if not collection.count():

    for _, rows in df.iterrows():
        collection.add(documents=rows["Techstack"],
                       metadatas={"links": rows["Links"]},
                       ids = [str(uuid.uuid4())])

In [19]:
links = collection.query(query_texts=["Experience in python", "Expertise in React"], n_results=2).get("metadatas")
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [28]:
job = json_res
job[0]['skills']

['Practical experience in AI and ML development with Python, TensorFlow, PyTorch, and cloud platforms (Azure, AWS, GCP)',
 'Hands-on experience building and fine-tuning LLMs, working with frameworks like LangChain, HuggingFace, or OpenAI APIs',
 'Experience designing MLOps pipelines, AI solution architecture, and deploying AI models in production',
 'Exposure to GenAI use cases such as document summarization, chatbots, intelligent assistants, and code generation',
 'Domain knowledge in one or more of the following: Government, Healthcare, Financial Services, Retail, Telecom',
 'Strong ability to translate complex business problems into AI solutions with measurable impact',
 'Knowledge of data privacy, ethical AI, and responsible AI principles']

In [31]:
job

[{'role': 'Consultant - AI Software Engineer - Digital FAAS - Abu Dhabi',
  'experience': '3 to 4 years of related work experience',
  'skills': ['Practical experience in AI and ML development with Python, TensorFlow, PyTorch, and cloud platforms (Azure, AWS, GCP)',
   'Hands-on experience building and fine-tuning LLMs, working with frameworks like LangChain, HuggingFace, or OpenAI APIs',
   'Experience designing MLOps pipelines, AI solution architecture, and deploying AI models in production',
   'Exposure to GenAI use cases such as document summarization, chatbots, intelligent assistants, and code generation',
   'Domain knowledge in one or more of the following: Government, Healthcare, Financial Services, Retail, Telecom',
   'Strong ability to translate complex business problems into AI solutions with measurable impact',
   'Knowledge of data privacy, ethical AI, and responsible AI principles'],
  'description': 'As an AI Engineering professional, you will help businesses unlock va

In [30]:
links = collection.query(query_texts=job[0]['skills'], n_results=2).get("metadatas")
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [32]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Business Value with AI Solutions - AtliQ Expertise

Dear [Recipient's Name],

I came across your job posting for a Consultant - AI Software Engineer - Digital FAAS - Abu Dhabi and was impressed by the requirements. AtliQ, an AI & Software Consulting company, specializes in facilitating the seamless integration of business processes through automated tools. Our expertise lies in empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

As an AI Engineering professional, you will help businesses unlock value by designing, building, and operationalizing scalable AI solutions using modern MLOps pipelines and cloud-native technologies. AtliQ has a proven track record in delivering such solutions, and I'd like to highlight a few relevant projects that demonstrate our capabilities:

- **MLOps Expertise**: Our team has extensive experience in designing and deploying AI models in production us